In [1]:
import glob
import pickle
import numpy as np
from numpy.testing import assert_allclose
import pandas as pd
from music21 import *
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, Bidirectional
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
from pygame import *

Using TensorFlow backend.


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [43]:
#creating an empty list to hold the notes in
notes = []
midi_scores = []

#this for loop goes through each midi file and flattens out the notes inside of it
for file in glob.glob("../data/training_melody/*.mid"):
    print(f'processing {file}')
    
    midi = converter.parse(file)
    midi_scores.append(midi)
    
    notes_to_parse = midi.flat.notes    
        
    for element in notes_to_parse: 
        if isinstance(element, note.Note): #if it's a single note, we don't have to join it to any other notes in the series
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord): #if it's a chord, we will have to join it to the other notes
            notes.append('.'.join(str(n) for n in element.normalOrder))
        else:
            print(element)

processing ../data/training_melody/ii-V9.mid
processing ../data/training_melody/c19.mid
processing ../data/training_melody/c25.mid
processing ../data/training_melody/Loop 8 (Toogood) B.mid
processing ../data/training_melody/c24.mid
processing ../data/training_melody/c30.mid
processing ../data/training_melody/c18.mid
processing ../data/training_melody/ii-V8.mid
processing ../data/training_melody/lofitrack_14_melody.mid
processing ../data/training_melody/lofitrack_3_melody.mid
processing ../data/training_melody/c9.mid
processing ../data/training_melody/c26.mid
processing ../data/training_melody/Loop 8 (Toogood) A.mid
processing ../data/training_melody/Loop 5 (Away With You).mid
processing ../data/training_melody/c27.mid
processing ../data/training_melody/c8.mid
processing ../data/training_melody/Cymatics - Eternity MIDI 11 - A Maj.mid
processing ../data/training_melody/Cymatics - Eternity MIDI 7 - E Min.mid
processing ../data/training_melody/tree9.mid
processing ../data/training_melody/c

processing ../data/training_melody/tree5.mid
processing ../data/training_melody/c13.mid
processing ../data/training_melody/lofitrack_5_melody.mid
processing ../data/training_melody/c12.mid
processing ../data/training_melody/Cymatics - Eternity MIDI 17 - A# Maj.mid
processing ../data/training_melody/tree4.mid
processing ../data/training_melody/c1.mid
processing ../data/training_melody/ii-V2.mid
processing ../data/training_melody/Cymatics - Eternity MIDI 19 - A# Min.mid
processing ../data/training_melody/Cymatics - Eternity MIDI 4 - D Maj.mid
processing ../data/training_melody/ii-V6.mid
processing ../data/training_melody/c5.mid
processing ../data/training_melody/c16.mid
processing ../data/training_melody/c17.mid
processing ../data/training_melody/tree1.mid
processing ../data/training_melody/Midi.mid
processing ../data/training_melody/c4.mid
processing ../data/training_melody/Loop 4 (Mantis).mid
processing ../data/training_melody/ii-V7.mid
processing ../data/training_melody/Cymatics - Ete

In [46]:
len(notes), len(set(notes))

(8839, 373)

In [19]:
""" Prepare the sequences used by the Neural Network """
sequence_length = 100

# get all pitch names
pitchnames = sorted(set(item for item in notes))

 # create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

network_input = []
network_output = []

# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])

n_patterns = len(network_input)

# reshape the input into a format compatible with LSTM layers
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
# normalize input
network_input = network_input / float(n_vocab)

network_output = np_utils.to_categorical(network_output)

In [21]:
""" create the structure of the neural network """
model = Sequential()
model.add(LSTM(
    512,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    recurrent_dropout=0.3,
    return_sequences=True
))
model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
model.add(LSTM(512))
model.add(BatchNorm())
model.add(Dropout(0.3))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(BatchNorm())
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [22]:
""" train the neural network """
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(
    filepath,
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)
callbacks_list = [checkpoint]

model.fit(network_input, network_output, epochs=200, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
1533/1533 [==============================] - 32s 21ms/step - loss: 5.9971
Epoch 2/20
1533/1533 [==============================] - 32s 21ms/step - loss: 5.9355
Epoch 3/20
1533/1533 [==============================] - 33s 21ms/step - loss: 5.8480
Epoch 4/20
1533/1533 [==============================] - 35s 23ms/step - loss: 5.7951
Epoch 5/20
1533/1533 [==============================] - 39s 25ms/step - loss: 5.6948
Epoch 6/20
1533/1533 [==============================] - 37s 24ms/step - loss: 5.7233
Epoch 7/20
1533/1533 [==============================] - 37s 24ms/step - loss: 5.6416
Epoch 8/20
1533/1533 [==============================] - 37s 24ms/step - loss: 5.6243
Epoch 9/20
1533/1533 [==============================] - 36s 24ms/step - loss: 5.6073
Epoch 10/20
1533/1533 [==============================] - 39s 25ms/step - loss: 5.5296
Epoch 11/20
1533/1533 [==============================] - 41s 27ms/step - loss: 5.4584
Epoch 12/20
1533/1533 [==============================] - 38s 25

In [34]:
""" Prepare the sequences used by the Neural Network """
# map between notes and integers and back
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

sequence_length = 100
network_input = []
output = []
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    output.append(note_to_int[sequence_out])

n_patterns = len(network_input)

# reshape the input into a format compatible with LSTM layers
normalized_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
# normalize input
normalized_input = normalized_input / float(n_vocab)

In [38]:
""" Generate notes from the neural network based on a sequence of notes """
# pick a random sequence from the input as a starting point for the prediction
start = np.random.randint(0, len(network_input)-1)

int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

pattern = network_input[start]
prediction_output = []

# generate 500 notes
for note_index in range(500):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)

    prediction = model.predict(prediction_input, verbose=0)

    index = np.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)

    pattern.append(index)
    pattern = pattern[1:len(pattern)]

In [39]:
""" convert the output from the prediction to notes and create a midi file
    from the notes """
offset = 0
output_notes = []

# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)

    # increase offset each iteration so that notes do not stack
    offset += 0.5

midi_stream = stream.Stream(output_notes)

In [40]:
output_notes

[<music21.note.Note F#>,
 <music21.note.Note F#>,
 <music21.note.Note F#>,
 <music21.note.Note F#>,
 <music21.note.Note F#>,
 <music21.note.Note F#>,
 <music21.note.Note F#>,
 <music21.note.Note F#>,
 <music21.note.Note F#>,
 <music21.note.Note F#>,
 <music21.note.Note F#>,
 <music21.note.Note F#>,
 <music21.note.Note F#>,
 <music21.note.Note F#>,
 <music21.note.Note F#>,
 <music21.note.Note F#>,
 <music21.note.Note F#>,
 <music21.note.Note F#>,
 <music21.note.Note F#>,
 <music21.note.Note F#>,
 <music21.note.Note F#>,
 <music21.note.Note F#>,
 <music21.note.Note F#>,
 <music21.note.Note F#>,
 <music21.note.Note F#>,
 <music21.note.Note F#>,
 <music21.note.Note F#>,
 <music21.note.Note F#>,
 <music21.note.Note F#>,
 <music21.note.Note F#>,
 <music21.note.Note F#>,
 <music21.note.Note F#>,
 <music21.note.Note F#>,
 <music21.note.Note F#>,
 <music21.note.Note F#>,
 <music21.note.Note F#>,
 <music21.note.Note F#>,
 <music21.note.Note F#>,
 <music21.note.Note F#>,
 <music21.note.Note F#>,


In [41]:
midi_stream.show('midi')